In [1]:
import pandas as pd

In [2]:
# loading datasets

df_llm = pd.read_csv('../PREP_llm_dataset.csv')

# data of the mme has to be the total 71M dataset (or maybe a subset of it, to see wheter it works).
# I will try it with a subset first (the 4M set)

#df_mme = pd.read_csv('../subset.csv')

In [4]:
""" # eigenlijk moet df_mme de 71M zijn

import pandas as pd

# Define the file path and desired chunk size (number of rows per batch)
file_path = '../SharedResponses.csv'
chunk_size = 100000  # Adjust based on memory constraints

# Initialize an empty list to store analysis results, if needed
mme71 = []

# Process the dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Perform your analysis on each chunk here
    # For example, you could compute summary statistics or apply transformations
    chunk_result = chunk.describe()  # Example of calculating summary statistics
    mme71.append(chunk_result)
    
# Combine results if necessary, or analyze each chunk separately
final_mme71 = pd.concat(mme71)
print(final_mme71)"""

" # eigenlijk moet df_mme de 71M zijn\n\nimport pandas as pd\n\n# Define the file path and desired chunk size (number of rows per batch)\nfile_path = '../SharedResponses.csv'\nchunk_size = 100000  # Adjust based on memory constraints\n\n# Initialize an empty list to store analysis results, if needed\nmme71 = []\n\n# Process the dataset in chunks\nfor chunk in pd.read_csv(file_path, chunksize=chunk_size):\n    # Perform your analysis on each chunk here\n    # For example, you could compute summary statistics or apply transformations\n    chunk_result = chunk.describe()  # Example of calculating summary statistics\n    mme71.append(chunk_result)\n    \n# Combine results if necessary, or analyze each chunk separately\nfinal_mme71 = pd.concat(mme71)\nprint(final_mme71)"

In [8]:
import pandas as pd
import numpy as np

# Define file paths and chunk size
file_path = '../SharedResponses.csv'
chunk_size = 500000

# Columns for matching
columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']

# Load the smaller dataset and calculate occurrences of each unique row
df_llm = pd.read_csv('../llm_125_dataset.csv')
df_llm_subset = df_llm[columns]
row_counts = df_llm_subset.groupby(columns).size().reset_index(name='count')

# Calculate the total count and sampling ratio
total_count = row_counts['count'].sum()
target_size = 1250000
row_counts['sample_size'] = (row_counts['count'] / total_count * target_size).round().astype(int)

In [9]:
# Initialize storage for the final sampled rows
mme71_matched_rows = []

count = 0

# Process the large dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    count += 1
    print(f'Processing chunk {count}...')

    # Merge current chunk with sample sizes to get only necessary rows
    merged_chunk = pd.merge(chunk, row_counts, on=columns, how='inner')

    # For each unique group, sample only the required number of rows based on 'sample_size'
    selected_rows = merged_chunk.groupby(columns, group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), x['sample_size'].iloc[0]), random_state=42)
    )

    # Drop 'count' and 'sample_size' columns as they are no longer needed
    selected_rows = selected_rows.drop(columns=['count', 'sample_size'])

    # Append sampled rows to the final list
    mme71_matched_rows.append(selected_rows)

    # Check if we've reached the target row count
    if sum(len(df) for df in mme71_matched_rows) >= target_size:
        print(f"Target of {target_size} rows reached.")
        break

# Concatenate selected rows to create the final subset, then limit to the target size
new_mme = pd.concat(mme71_matched_rows).reset_index(drop=True).head(target_size)

# Final shape check
print(new_mme.shape)

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\1988205066.py:7: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 4...
Processing chunk 5...
Target of 1250000 rows reached.
(1250000, 41)


In [10]:
new_mme

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
0,2CC4NLDWqoN7zbPkB,-1651116233_8862961275634861.0,8.862961e+15,11,0,0,0,0,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2ZC78irCvphjcaGGe,-1334040459_6793492690194398.0,6.793493e+15,8,0,0,0,0,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25zKASnoN2h6NiML9,-1842858075_1296421467,1.296421e+09,4,0,0,0,0,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2YL3aZWPJ5P2FpmWy,1245245431_724207572155601.0,7.242076e+14,2,0,0,0,0,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23vtiHkN2XptMJoqn,2105421602_9590344519820064.0,9.590345e+15,8,0,0,0,0,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249995,5Jj3TxrnTpTC2tYiW,-283815108_942993251,9.429933e+08,2,0,0,0,0,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1249996,5YpfZB6HwFKPymobX,-2129604404_9832953594082390.0,9.832954e+15,11,0,0,0,0,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1249997,5ggdeWcKkomq8XeH3,-1019657361_2617799162.0,2.617799e+09,1,0,0,0,0,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1249998,5L6sQ8vgFqXev9qKp,-111190951_933841399976875.0,9.338414e+14,10,0,0,0,0,Female,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
new_mme_subset = new_mme[columns]
print(new_mme_subset.shape)

(1250000, 27)


In [12]:
for col in df_llm_subset.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = new_mme[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

Mismatch in distribution for 'PedPed'.
Original distribution:
 PedPed
0    0.502644
1    0.497356
Name: proportion, dtype: float64
Subset distribution:
 PedPed
0    0.512296
1    0.487704
Name: proportion, dtype: float64
Mismatch in distribution for 'Barrier'.
Original distribution:
 Barrier
0    0.748678
1    0.251322
Name: proportion, dtype: float64
Subset distribution:
 Barrier
0    0.788711
1    0.211289
Name: proportion, dtype: float64
Mismatch in distribution for 'CrossingSignal'.
Original distribution:
 CrossingSignal
0    0.622715
2    0.188958
1    0.188327
Name: proportion, dtype: float64
Subset distribution:
 CrossingSignal
0    0.606266
2    0.239281
1    0.154453
Name: proportion, dtype: float64
Mismatch in distribution for 'AttributeLevel'.
Original distribution:
 AttributeLevel
Young      0.086392
Old        0.086392
Hoomans    0.085493
Pets       0.085493
Less       0.083838
More       0.083838
Fat        0.082443
Fit        0.082443
Male       0.081512
Female     0.081

# ----------------------------------

In [17]:
import pandas as pd
import numpy as np

# Define file paths and chunk size
file_path = '../SharedResponses.csv'
chunk_size = 100000

# Columns for matching
columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']

# Load the smaller dataset and calculate occurrences of each unique row
df_llm = pd.read_csv('../PREP_llm_dataset.csv')
df_llm_subset = df_llm[columns]
row_counts = df_llm_subset.groupby(columns).size().reset_index(name='count')
print(row_counts)       # rowcounts = df with all unique rows in the df_llm and the count of each row in the 'count' column

# count in this df is the sample size of those rows in the df_mme df


       PedPed  Barrier  CrossingSignal AttributeLevel ScenarioTypeStrict  \
0           0        0               0            Fat            Fitness   
1           0        0               0            Fat            Fitness   
2           0        0               0            Fat            Fitness   
3           0        0               0            Fat            Fitness   
4           0        0               0            Fat            Fitness   
...       ...      ...             ...            ...                ...   
34235       1        0               2          Young                Age   
34236       1        0               2          Young                Age   
34237       1        0               2          Young                Age   
34238       1        0               2          Young                Age   
34239       1        0               2          Young                Age   

       NumberOfCharacters  DiffNumberOFCharacters  Man  Woman  Pregnant  ...  \
0      

In [ ]:
""" # Initialize storage for the final sampled rows
mme71_matched_rows = []
target_size = 1250000

count = 0

# Process the large dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    count += 1
    print(f'Processing chunk {count}...')

    # Merge current chunk with sample sizes to get only necessary rows
    merged_chunk = pd.merge(chunk, row_counts, on=columns, how='inner')

    print(merged_chunk)

    # For each unique group, sample only the required number of rows based on 'count'
    selected_rows = merged_chunk.groupby(columns, group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), x['count'].iloc[0]), random_state=42)
    )
    print(selected_rows)

    # Drop 'count' and 'sample_size' columns as they are no longer needed
    selected_rows = selected_rows.drop(columns=['count'])

    # Append sampled rows to the final list
    mme71_matched_rows.append(selected_rows)

    # Check if we've reached the target row count
    if sum(len(df) for df in mme71_matched_rows) >= target_size:
        print(f"Target of {target_size} rows reached.")
        break

# Concatenate selected rows to create the final subset, then limit to the target size
new_mme = pd.concat(mme71_matched_rows).reset_index(drop=True)

# Final shape check
print(new_mme.shape)"""

Processing chunk 1...
              ResponseID              ExtendedSessionID        UserID  \
0      2222bRQqBTZ6dLnPH    32757157_6999801415950060.0  6.999801e+15   
1      23n3vnNRxg9rzTNbC         346750175_2526897339.0  2.526897e+09   
2      24FYeMfaP79PENS2o        -803509540_2562185549.0  2.562186e+09   
3      28TBdHDpgWdcttWAs          -1120404289_110062323  1.100623e+08   
4      29tXzqHD7jCuCcnkF  -1904655469_495495102368295.0  4.954951e+14   
...                  ...                            ...           ...   
66184  2AZAmPTWpkZqo53ck  1499521330_3693604043225167.0  3.693604e+15   
66185  2AZCrhSvn8BSStsjE          1719023912_1134492437  1.134492e+09   
66186  2AZGbg2ZFaknEgsNx        1252720589_3306455966.0  3.306456e+09   
66187  2AZYT2v6qDPwxLbm2   -778614065_466909381529543.0  4.669094e+14   
66188  2AZZoSWxKEWLc99Qr    957672980_698582830892113.0  6.985828e+14   

       ScenarioOrder  Intervention  PedPed  Barrier  CrossingSignal  \
0                  7          

C:\Users\esmku\AppData\Local\Temp\ipykernel_19920\2054791189.py:8: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 19...
              ResponseID               ExtendedSessionID        UserID  \
0      4pjwBsKE2tmomqABo    308204827_3142920609097928.0  3.142921e+15   
1      4udnmerwPPHP5AjGd  -1669937293_6821851419361860.0  6.821851e+15   
2      4pjwMBz4MzEHrHLR4          838294406_3473597831.0  3.473598e+09   
3      4pkSmNGFKdJtAGsvP   1587472350_3778052747115207.0  3.778053e+15   
4      4pn3aWWuMCkgLxcXC        -1877005271_2584402399.0  2.584402e+09   
...                  ...                             ...           ...   
55151  4yHjrqyq3LNfjX4QA  -2052546367_8530682776081397.0  8.530683e+15   
55152  4yHkBy8orEE8xWs38   -184937267_2257553179685025.0  2.257553e+15   
55153  4yHsMuiaR87s8sHTg         1737969093_2548399257.0  2.548399e+09   
55154  4yHt9YjA5Yfhpm6ru             1434881712_52817232  5.281723e+07   
55155  4yJJemrNBAuPogoLZ   1457664276_7018738569705645.0  7.018739e+15   

       ScenarioOrder  Intervention  PedPed  Barrier  CrossingSignal  \
0                

In [ ]:
# Initialize a list to collect matched rows from each chunk
mme71_matched_rows = []
count = 0
cumulative_row_count = 0  # Tracks the cumulative number of rows added

# Process the large dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    count += 1
    print(f'Processing chunk {count}...')

    # Merge the current chunk with row counts to get only necessary rows
    merged_chunk = pd.merge(chunk, row_counts, on=columns, how='inner')

    # For each unique group, sample only the required number of rows based on 'count'
    selected_rows = merged_chunk.groupby(columns, group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), x['count'].iloc[0]), random_state=42)
    )

    # Drop 'count' column as it’s no longer needed
    selected_rows = selected_rows.drop(columns=['count'])

    # Append sampled rows to the list, limiting to target size
    cumulative_row_count += len(selected_rows)
    mme71_matched_rows.append(selected_rows)

    # Check if the cumulative count has reached or exceeded the target size
    if cumulative_row_count >= target_size:
        print(f"Target of {target_size} rows reached.")
        break

# Concatenate the selected rows and trim to the exact target size if needed
new_mme = pd.concat(mme71_matched_rows, ignore_index=True).head(target_size)

# Final shape check and distribution validation
print(new_mme.shape)

# Check distribution to ensure it matches original
final_distribution = new_mme['ScenarioTypeStrict'].value_counts(normalize=True)
print("Final subset distribution:\n", final_distribution)

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...


C:\Users\esmku\AppData\Local\Temp\ipykernel_19920\1972013304.py:7: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 19...
Processing chunk 20...
Target of 1250000 rows reached.
(1250000, 41)
Final subset distribution:
 ScenarioTypeStrict
Age              0.243631
Fitness          0.227446
Gender           0.215914
Species          0.183017
Social Status    0.078772
Utilitarian      0.051220
Name: proportion, dtype: float64


In [19]:
new_mme_27 = new_mme[columns]
print(new_mme_27.shape)

# Create crosstabs for each column and compare
for col in df_llm_subset.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = new_mme_27[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

(1250000, 27)
Mismatch in distribution for 'PedPed'.
Original distribution:
 PedPed
0    0.502728
1    0.497272
Name: proportion, dtype: float64
Subset distribution:
 PedPed
0    0.516906
1    0.483094
Name: proportion, dtype: float64
Mismatch in distribution for 'Barrier'.
Original distribution:
 Barrier
0    0.748636
1    0.251364
Name: proportion, dtype: float64
Subset distribution:
 Barrier
0    0.793641
1    0.206359
Name: proportion, dtype: float64
Mismatch in distribution for 'CrossingSignal'.
Original distribution:
 CrossingSignal
0    0.622809
2    0.188983
1    0.188208
Name: proportion, dtype: float64
Subset distribution:
 CrossingSignal
0    0.601477
2    0.243826
1    0.154697
Name: proportion, dtype: float64
Mismatch in distribution for 'AttributeLevel'.
Original distribution:
 AttributeLevel
Young      0.086332
Old        0.086332
Hoomans    0.085502
Pets       0.085502
Less       0.083888
More       0.083888
Fat        0.082395
Fit        0.082395
Male       0.081562
Fe

# ---------------------------------------------------

In [13]:
import pandas as pd
import numpy as np

# Define file paths and chunk size
file_path = '../SharedResponses.csv'
chunk_size = 500000

# Columns for matching
columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']

# Load the smaller dataset and calculate occurrences of each unique row
df_llm = pd.read_csv('../llm_125_dataset.csv')
df_llm_subset = df_llm[columns]
row_counts = df_llm_subset.groupby(columns).size().reset_index(name='count')

# Calculate the total count and sampling ratio
total_count = row_counts['count'].sum()
target_size = 1250000
row_counts['sample_size'] = (row_counts['count'] / total_count * target_size).round().astype(int)

# Initialize storage for the final sampled rows
mme71_matched_rows = []

count = 0

# Process the large dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    count += 1
    print(f'Processing chunk {count}...')

    # Merge current chunk with sample sizes to get only necessary rows
    merged_chunk = pd.merge(chunk, row_counts, on=columns, how='inner')

    # For each unique group, sample only the required number of rows based on 'sample_size'
    selected_rows = merged_chunk.groupby(columns, group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), x['sample_size'].iloc[0]), random_state=42)
    )

    # Drop 'count' and 'sample_size' columns as they are no longer needed
    selected_rows = selected_rows.drop(columns=['count', 'sample_size'])

    # Append sampled rows to the final list
    mme71_matched_rows.append(selected_rows)

    # Check if we've reached the target row count
    if sum(len(df) for df in mme71_matched_rows) >= target_size:
        print(f"Target of {target_size} rows reached.")
        break

# Concatenate selected rows to create the final subset, then limit to the target size
new_mme = pd.concat(mme71_matched_rows).reset_index(drop=True).head(target_size)

# Final shape check
print(new_mme.shape)


Processing chunk 1...
Processing chunk 2...
Processing chunk 3...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\440102282.py:32: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 4...
Processing chunk 5...
Target of 1250000 rows reached.
(1250000, 41)


# ------------------------------ 

In [18]:
import pandas as pd

# Define paths and columns for matching
file_path = '../SharedResponses.csv'
chunk_size = 500000  # Adjust this based on memory limits

# Columns for matching
columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']

# Load and preprocess the LLM dataset to get the exact counts of each unique row
df_llm = pd.read_csv('../PREP_llm_dataset.csv')
df_llm_subset = df_llm[columns]
row_counts = df_llm_subset.value_counts().reset_index(name='count')

# Convert row_counts to a dictionary for faster lookup of required counts
required_counts = row_counts.set_index(columns)['count'].to_dict()

In [ ]:
# Initialize a dictionary to accumulate matched rows and track counts
accumulated_rows = {key: [] for key in required_counts.keys()}
accumulated_counts = {key: 0 for key in required_counts.keys()}

count = 0

# Process the MME dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    count += 1
    print(f'Processing chunk {count}...')
    # Merge the chunk with row_counts on the matching columns
    merged_chunk = pd.merge(chunk, row_counts, on=columns, how='inner')

    # For each unique row combination in the merged chunk, sample the required number of rows
    for row_key, group in merged_chunk.groupby(columns):
        # Get the remaining count needed for this unique combination
        required_count = required_counts.get(tuple(row_key), 0)
        accumulated_count = accumulated_counts.get(tuple(row_key), 0)
        rows_needed = required_count - accumulated_count

        if rows_needed > 0:
            # Sample only as many rows as needed
            sampled_rows = group.sample(n=min(len(group), rows_needed), random_state=42)
            accumulated_rows[row_key].append(sampled_rows)

            # Update the accumulated count
            accumulated_counts[row_key] += len(sampled_rows)

    # Early stopping if all required rows are accumulated
    if all(accumulated_counts[key] >= required_counts[key] for key in required_counts):
        print("All required rows have been accumulated.")
        break

# Concatenate all accumulated rows to create the final DataFrame
final_matched_rows = pd.concat(
    [pd.concat(accumulated_rows[key]) for key in accumulated_rows if accumulated_rows[key]],
    ignore_index=True
)

# Final shape check
print(final_matched_rows.shape)

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\155003068.py:8: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\155003068.py:8: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...
Processing chunk 29...
Processing chunk 30...
Processing chunk 31...
Processing chunk 32...
Processing chunk 33...
Processing chunk 34...
Processing chunk 35...
Processing chunk 36...
Processing chunk 37...
Processing chunk 38...
Processing chunk 39...
Processing chunk 40...
Processing chunk 41...
Processing chunk 42...
Processing chunk 43...
Processing chunk 44...
Processing chunk 45...
Processing chunk 46...
Processing chunk 47...
Processing chunk 48...
Processing chunk 49...
Processing chunk 50...
Processing chunk 51...
Processing chunk 52...
Processing chunk 53...
Processing chunk 54...
Processing chunk 55...
Processing chunk 56...
Processing 

...

Processing chunk 81...
Processing chunk 82...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\155003068.py:8: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 83...
Processing chunk 84...
Processing chunk 85...
Processing chunk 86...
Processing chunk 87...
Processing chunk 88...
Processing chunk 89...
Processing chunk 90...
Processing chunk 91...
Processing chunk 92...
Processing chunk 93...
Processing chunk 94...
Processing chunk 95...
Processing chunk 96...
Processing chunk 97...
Processing chunk 98...
Processing chunk 99...
Processing chunk 100...
Processing chunk 101...
Processing chunk 102...
Processing chunk 103...
Processing chunk 104...
Processing chunk 105...
Processing chunk 106...
Processing chunk 107...
Processing chunk 108...
Processing chunk 109...
Processing chunk 110...
Processing chunk 111...
Processing chunk 112...
Processing chunk 113...
Processing chunk 114...
Processing chunk 115...
Processing chunk 116...
Processing chunk 117...
Processing chunk 118...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\155003068.py:8: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 119...
Processing chunk 120...
Processing chunk 121...
Processing chunk 122...
Processing chunk 123...
Processing chunk 124...
Processing chunk 125...
Processing chunk 126...
Processing chunk 127...
Processing chunk 128...
Processing chunk 129...
Processing chunk 130...
Processing chunk 131...
Processing chunk 132...
Processing chunk 133...
Processing chunk 134...
Processing chunk 135...
Processing chunk 136...
Processing chunk 137...
Processing chunk 138...
Processing chunk 139...
Processing chunk 140...
Processing chunk 141...
(977520, 42)


In [20]:
final_matched_rows_sub = final_matched_rows[columns]
print(final_matched_rows_sub.shape)

(977520, 27)


In [21]:
# Create crosstabs for each column and compare
for col in df_llm_subset.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = final_matched_rows_sub[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

Mismatch in distribution for 'PedPed'.
Original distribution:
 PedPed
0    0.502728
1    0.497272
Name: proportion, dtype: float64
Subset distribution:
 PedPed
0    0.501007
1    0.498993
Name: proportion, dtype: float64
Mismatch in distribution for 'Barrier'.
Original distribution:
 Barrier
0    0.748636
1    0.251364
Name: proportion, dtype: float64
Subset distribution:
 Barrier
0    0.744633
1    0.255367
Name: proportion, dtype: float64
Mismatch in distribution for 'CrossingSignal'.
Original distribution:
 CrossingSignal
0    0.622809
2    0.188983
1    0.188208
Name: proportion, dtype: float64
Subset distribution:
 CrossingSignal
0    0.629825
2    0.186193
1    0.183982
Name: proportion, dtype: float64
Mismatch in distribution for 'AttributeLevel'.
Original distribution:
 AttributeLevel
Young      0.086332
Old        0.086332
Hoomans    0.085502
Pets       0.085502
Less       0.083888
More       0.083888
Fat        0.082395
Fit        0.082395
Male       0.081562
Female     0.081

# -----------------------------------------------------------

In [22]:
import pandas as pd
import numpy as np

# Define paths and columns for matching
file_path = '../SharedResponses.csv'
chunk_size = 500000  # Adjust based on memory constraints
target_total_rows = 1250000  # Total target rows

# Columns for matching
columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']

# Load the LLM dataset and calculate required counts
df_llm = pd.read_csv('../PREP_llm_dataset.csv')
df_llm_subset = df_llm[columns]

# Calculate the exact count per unique row in LLM dataset
target_row_counts = df_llm_subset.value_counts().reset_index(name='target_count')

# Initialize dictionaries to accumulate rows and counts
accumulated_rows = {key: [] for key in target_row_counts.set_index(columns).index}
accumulated_counts = {key: 0 for key in target_row_counts.set_index(columns).index}

In [ ]:
# Initialize a counter to track total number of rows accumulated
total_rows = 0
count = 0

# Process the MME dataset in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    count += 1
    print(f'Processing chunk {count}...')

    # Merge the chunk with target_row_counts on the matching columns
    merged_chunk = pd.merge(chunk, target_row_counts, on=columns, how='inner')

    # For each unique row combination in the merged chunk, sample required number of rows
    for row_key, group in merged_chunk.groupby(columns):
        target_count = target_row_counts.set_index(columns).at[tuple(row_key), 'target_count']
        accumulated_count = accumulated_counts.get(tuple(row_key), 0)
        rows_needed = target_count - accumulated_count

        if rows_needed > 0:
            # Sample only the required rows to match distribution
            sampled_rows = group.sample(n=min(len(group), rows_needed), random_state=42)
            accumulated_rows[row_key].append(sampled_rows)

            # Update counts
            num_sampled_rows = len(sampled_rows)
            accumulated_counts[row_key] += num_sampled_rows
            total_rows += num_sampled_rows

            # Early stopping if we reach the total row limit
            if total_rows >= target_total_rows:
                print("Target of 1.25 million rows has been reached.")
                break

    # Check again to break the outer loop if the target is met
    if total_rows >= target_total_rows:
        break

# Concatenate accumulated rows and limit to 1.25 million rows if slightly exceeded
final_matched_rows = pd.concat(
    [pd.concat(accumulated_rows[key]) for key in accumulated_rows if accumulated_rows[key]],
    ignore_index=True
).head(target_total_rows)

# Validate distribution to ensure it matches the original LLM dataset
final_distribution = final_matched_rows['ScenarioTypeStrict'].value_counts(normalize=True)
print("Final subset distribution:\n", final_distribution)

# Check final shape
print(final_matched_rows.shape)


Processing chunk 1...
Processing chunk 2...
Processing chunk 3...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\999150989.py:6: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...


C:\Users\esmku\AppData\Local\Temp\ipykernel_2088\999150989.py:6: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):


Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...


# ------------------------------------------------------

In [13]:
final_matched_rows_sub = final_matched_rows[columns]
print(final_matched_rows_sub.shape)

# Create crosstabs for each column and compare
for col in df_llm_subset.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = final_matched_rows_sub[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

NameError: name 'final_matched_rows' is not defined

In [24]:
# specifying which columns have to be the same
# all columns that are not meaningful for the decision - so only the scenario typing columns

# these not: 'ResponseID', 'UserID', 'Intervention', 'Saved'

columns = ['PedPed', 'Barrier',
       'CrossingSignal', 'AttributeLevel', 'ScenarioTypeStrict',
       'NumberOfCharacters', 'DiffNumberOFCharacters', 'Man', 'Woman',
       'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
       'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
       'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
       'MaleDoctor', 'Dog', 'Cat']
print(len(columns))
print(df_llm.shape)

27
(1547132, 31)


In [86]:
# creating a subset of the datasets so that only the relevant columns are included and taken for the comparison

df_llm_subset = df_llm[columns]
print(df_llm_subset.shape)

df_mme_subset = df_mme[columns]
print(df_mme_subset.shape)

(1547132, 27)
(4000000, 27)


In [ ]:
# checking the unique scenario's of the llm dataset

print(df_llm_subset.drop_duplicates().shape)

(34394, 27)


In [87]:
import pandas as pd

# Step 1: Count occurrences of each unique row in A (df_llm_subset)
row_counts = df_llm_subset.groupby(list(df_llm_subset.columns)).size().reset_index(name='count')

In [88]:
# Step 2: Perform a merge with B (df_mme_subset) on all columns
merged = pd.merge(df_mme_subset, row_counts, on=list(df_llm_subset.columns), how='inner')

In [89]:
# Step 3: Within each unique row group, keep only the required number of rows specified by the 'count' column
# This involves grouping by all columns except 'count' and then taking head(n) based on the 'count' value
result = merged.groupby(list(df_llm_subset.columns), group_keys=False).apply(lambda x: x.head(x['count'].iloc[0]))

In [90]:
# Drop the 'count' column as it's no longer needed
new_mme = result.drop(columns=['count']).reset_index(drop=True)

print(new_mme.shape)

(1036882, 27)


In [105]:
# Count occurrences of each unique row in both datasets
llm_counts = df_llm_subset.groupby(list(df_llm_subset.columns)).size().reset_index(name='count_llm')
mme_counts = df_mme_subset.groupby(list(df_llm_subset.columns)).size().reset_index(name='count_mme')

# Merge these counts to identify potential mismatches
count_check = pd.merge(llm_counts, mme_counts, on=list(df_llm_subset.columns), how='left').fillna(0)

# Find rows where mme does not have enough rows to match llm
insufficient_matches = count_check[count_check['count_llm'] > count_check['count_mme']]
if not insufficient_matches.empty:
    print("Warning: Insufficient matching rows for the following patterns:\n", insufficient_matches)
else:
    print("All unique rows in LLM have sufficient matches in MME.")


        PedPed  Barrier  CrossingSignal AttributeLevel ScenarioTypeStrict  \
16          0        0               0            Fat            Fitness   
34          0        0               0            Fat            Fitness   
35          0        0               0            Fat            Fitness   
36          0        0               0            Fat            Fitness   
37          0        0               0            Fat            Fitness   
...       ...      ...             ...            ...                ...   
34350       1        0               2          Young                Age   
34351       1        0               2          Young                Age   
34352       1        0               2          Young                Age   
34353       1        0               2          Young                Age   
34354       1        0               2          Young                Age   

       NumberOfCharacters  DiffNumberOFCharacters  Man  Woman  Pregnant  ...  \
16    

In [15]:
new_mme[columns]
new_mme.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

In [16]:
# Create crosstabs for each column and compare
for col in df_llm_subset.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = new_mme[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

Mismatch in distribution for 'PedPed'.
Original distribution:
 PedPed
0    0.502644
1    0.497356
Name: proportion, dtype: float64
Subset distribution:
 PedPed
0    0.512296
1    0.487704
Name: proportion, dtype: float64
Mismatch in distribution for 'Barrier'.
Original distribution:
 Barrier
0    0.748678
1    0.251322
Name: proportion, dtype: float64
Subset distribution:
 Barrier
0    0.788711
1    0.211289
Name: proportion, dtype: float64
Mismatch in distribution for 'CrossingSignal'.
Original distribution:
 CrossingSignal
0    0.622715
2    0.188958
1    0.188327
Name: proportion, dtype: float64
Subset distribution:
 CrossingSignal
0    0.606266
2    0.239281
1    0.154453
Name: proportion, dtype: float64
Mismatch in distribution for 'AttributeLevel'.
Original distribution:
 AttributeLevel
Young      0.086392
Old        0.086392
Hoomans    0.085493
Pets       0.085493
Less       0.083838
More       0.083838
Fat        0.082443
Fit        0.082443
Male       0.081512
Female     0.081

### Exampled data

In [63]:
# Example A and B datasets
A = pd.DataFrame({
    'col1': ['A', 'B', 'C', 'A', 'B'],
    'col2': [1, 2, 3, 1, 2],
    'col3': ['X', 'Y', 'Z', 'X', 'Y']
})

print(A)

B = pd.DataFrame({
    'col1': ['A', 'B', 'C', 'A', 'B', 'A', 'B', 'C'],
    'col2': [1, 2, 3, 1, 2, 1, 2, 3],
    'col3': ['X', 'Y', 'Z', 'X', 'Y', 'X', 'Y', 'Z']
})

print(B)

import pandas as pd

# Sample data for illustration
# df_llm_subset = pd.DataFrame(...)  == A
# df_mme_subset = pd.DataFrame(...)  == B

# Step 1: Count occurrences of each unique row in A (df_llm_subset)
row_counts = A.groupby(list(A.columns)).size().reset_index(name='count')

# Step 2: Perform a merge with B (df_mme_subset) on all columns
merged = pd.merge(B, row_counts, on=list(A.columns), how='inner')

# Step 3: Within each unique row group, keep only the required number of rows specified by the 'count' column
# This involves grouping by all columns except 'count' and then taking head(n) based on the 'count' value
result = merged.groupby(list(A.columns), group_keys=False).apply(lambda x: x.head(x['count'].iloc[0]))

# Drop the 'count' column as it's no longer needed
result = result.drop(columns=['count']).reset_index(drop=True)

print(result)



  col1  col2 col3
0    A     1    X
1    B     2    Y
2    C     3    Z
3    A     1    X
4    B     2    Y
  col1  col2 col3
0    A     1    X
1    B     2    Y
2    C     3    Z
3    A     1    X
4    B     2    Y
5    A     1    X
6    B     2    Y
7    C     3    Z
  col1  col2 col3
0    A     1    X
1    A     1    X
2    B     2    Y
3    B     2    Y
4    C     3    Z


In [65]:
# Create crosstabs for each column and compare
for col in A.columns:
    original_dist = A[col].value_counts(normalize=True)
    subset_dist = result[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

The distribution for 'col1' matches.
The distribution for 'col2' matches.
The distribution for 'col3' matches.


In [100]:
import pandas as pd

In [102]:
x = pd.read_csv('../rsos231393_si_001.csv')



In [103]:
x.shape

(260, 2)

,Scenario,Answer
0,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...
1,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should not swerve into the ...
2,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...
3,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...
4,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should not swerve and hit t...
...,...,...
255,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...
256,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...
257,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...
258,Case 1.\nA self-driving car with sudden brake ...,Case 2.\n\nThe car should continue ahead and d...


# ------------------------------------------------------------------------

### Doing it first the one way around (so taking a subset of the MME based on the LLM)
This subset has then all the possible rows present in A (LLM) that are also present in B (MME)
It appears that this is not all of them (as was suspected as the full dataset is 71M), but rather 977535 rows

I now want to do the same code but the other way around. This will result in a dataset (subset of the llm dataset) that has also 977535 rows (as I know for sure that these are all in there), with the same distribution as the MME subset just taken. This means however that the distribution isn't perfectly divided. But is this bad? Not really - would rather want the distribution between both of the LLM and the MME to be the same ipv the total distribution to be nice and orderned.

In [44]:
""" import pandas as pd

# Columns for matching
columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']

# Load your dataset A
dataset_llm = pd.read_csv('../PREP_llm_dataset.csv')
dataset_llm[columns]

# Let even op - je hebt hier NIET deze gebruikt om de functie te runnen!"""

" import pandas as pd\n\n# Columns for matching\ncolumns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', \n           'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', \n           'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', \n           'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', \n           'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', \n           'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']\n\n# Load your dataset A\ndataset_llm = pd.read_csv('../PREP_llm_dataset.csv')\ndataset_llm[columns]\n\n# Let even op - je hebt hier NIET deze gebruikt om de functie te runnen!"

In [43]:
"""# Define the function to match rows in dataset B by loading in chunks
def subset_dataset_mme_to_match_llm(dataset_llm, dataset_mme, chunk_size=500000):

    # Create a dictionary to store counts of each unique row from dataset A
    row_counts_llm = dataset_llm.value_counts().to_dict()

    # Initialize an empty DataFrame to store matching rows from dataset B
    subset_mme = pd.DataFrame(columns=dataset_llm.columns)

    # Read dataset B in chunks
    for chunk in pd.read_csv(dataset_mme, chunksize=chunk_size):
        # Filter rows in the chunk that are present in dataset A (match by rows)
        matching_rows = chunk[chunk.isin(dataset_llm.to_dict(orient='list')).all(axis=1)]
        
        # Check the number of times each row appears in dataset A and only retain the exact counts
        matching_rows = matching_rows[matching_rows.apply(tuple, axis=1).map(row_counts_llm).fillna(0) > 0]
        
        # Decrease count for each row in dataset A that was matched in dataset B
        for row in matching_rows.itertuples(index=False):
            row_counts_llm[row] -= 1

            # Append to the final subset if count is positive
            if row_counts_llm[row] >= 0:
                subset_mme = pd.concat([subset_mme, pd.DataFrame([row._asdict()])], ignore_index=True)
        
        # If we have collected all rows, break early to save time and memory
        if len(subset_mme) == len(dataset_llm):
            break
    
    # Final check to make sure the number of rows match
    if len(subset_mme) != len(dataset_llm):
        raise ValueError("The subset of dataset B does not match the required row count from dataset A.")

    return subset_mme"""

'# Define the function to match rows in dataset B by loading in chunks\ndef subset_dataset_mme_to_match_llm(dataset_llm, dataset_mme, chunk_size=500000):\n\n    # Create a dictionary to store counts of each unique row from dataset A\n    row_counts_llm = dataset_llm.value_counts().to_dict()\n\n    # Initialize an empty DataFrame to store matching rows from dataset B\n    subset_mme = pd.DataFrame(columns=dataset_llm.columns)\n\n    # Read dataset B in chunks\n    for chunk in pd.read_csv(dataset_mme, chunksize=chunk_size):\n        # Filter rows in the chunk that are present in dataset A (match by rows)\n        matching_rows = chunk[chunk.isin(dataset_llm.to_dict(orient=\'list\')).all(axis=1)]\n        \n        # Check the number of times each row appears in dataset A and only retain the exact counts\n        matching_rows = matching_rows[matching_rows.apply(tuple, axis=1).map(row_counts_llm).fillna(0) > 0]\n        \n        # Decrease count for each row in dataset A that was matche

In [ ]:
"""# Create the subset
new_mme_27 = subset_dataset_mme_to_match_llm(dataset_llm, '../SharedResponses.csv')

# Check distribution
for col in dataset_llm.columns:
    original_dist = dataset_llm[col].value_counts(normalize=True)
    subset_dist = new_mme_27[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)
"""

In [ ]:
import pandas as pd
from collections import defaultdict

def create_matching_subset(dataset_a_path, dataset_b_path, chunk_size=10000):
    # Define columns to match on
    columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
               'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
               'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
               'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
               'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
               'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']
    
    # Load dataset A and count occurrences of each unique row configuration
    df_a = pd.read_csv(dataset_a_path, usecols=columns)
    a_counts = df_a.groupby(columns).size().to_dict()

    # Dictionary to store matched rows with the exact distribution
    matching_rows = defaultdict(list)
    final_matching_subset_b = pd.DataFrame(columns=columns)
    
    z = 0
    
    # Process B in chunks to avoid memory overload
    for chunk in pd.read_csv(dataset_b_path, chunksize=chunk_size, usecols=columns):
        z += 1

        print(f"Processing chunk...{z}")

        # Group each chunk by unique row configurations
        b_counts = chunk.groupby(columns).size().to_dict()
        
        # Loop through each unique row configuration in A
        for row_config, required_count in a_counts.items():
            if required_count <= 0:
                continue  # Skip if we've already gathered all required rows for this configuration
            
            # Check if the row configuration exists in the current chunk
            if row_config in b_counts:
                # Select matching rows from the chunk
                matching_chunk_rows = chunk[(chunk[columns] == pd.Series(row_config, index=columns)).all(axis=1)]
                
                # Select only the required number of rows
                selected_rows = matching_chunk_rows.iloc[:required_count]
                
                # Append selected rows to the final dataframe
                final_matching_subset_b = pd.concat([final_matching_subset_b, selected_rows], ignore_index=True)
                
                "print(final_matching_subset_b.shape)"
                
                # Decrease the required count
                a_counts[row_config] -= len(selected_rows)

        # Stop if we've collected enough rows to match all counts in A
        if sum(a_counts.values()) == 0:
            break
    
    return final_matching_subset_b

In [30]:
# Usage
df_a_path = '../PREP_llm_dataset.csv'
df_b_path = '../SharedResponses.csv'
new_mme_27 = create_matching_subset(df_a_path, df_b_path)

Processing chunk...1
Processing chunk...2
Processing chunk...3
Processing chunk...4
Processing chunk...5
Processing chunk...6
Processing chunk...7
Processing chunk...8
Processing chunk...9
Processing chunk...10
Processing chunk...11
Processing chunk...12
Processing chunk...13
Processing chunk...14
Processing chunk...15
Processing chunk...16
Processing chunk...17
Processing chunk...18
Processing chunk...19
Processing chunk...20
Processing chunk...21
Processing chunk...22
Processing chunk...23
Processing chunk...24
Processing chunk...25
Processing chunk...26
Processing chunk...27
Processing chunk...28
Processing chunk...29
Processing chunk...30
Processing chunk...31
Processing chunk...32
Processing chunk...33
Processing chunk...34
Processing chunk...35
Processing chunk...36
Processing chunk...37
Processing chunk...38
Processing chunk...39
Processing chunk...40
Processing chunk...41
Processing chunk...42
Processing chunk...43
Processing chunk...44
Processing chunk...45
Processing chunk...

In [31]:
# Verify distribution
df_llm_subset = pd.read_csv(df_a_path)  # Reload A to verify distribution
for col in new_mme_27.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = new_mme_27[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

Mismatch in distribution for 'PedPed'.
Original distribution:
 PedPed
0    0.502728
1    0.497272
Name: proportion, dtype: float64
Subset distribution:
 PedPed
0    0.501005
1    0.498995
Name: proportion, dtype: float64
Mismatch in distribution for 'Barrier'.
Original distribution:
 Barrier
0    0.748636
1    0.251364
Name: proportion, dtype: float64
Subset distribution:
 Barrier
0    0.744636
1    0.255364
Name: proportion, dtype: float64
Mismatch in distribution for 'CrossingSignal'.
Original distribution:
 CrossingSignal
0    0.622809
2    0.188983
1    0.188208
Name: proportion, dtype: float64
Subset distribution:
 CrossingSignal
0    0.629821
2    0.186199
1    0.183980
Name: proportion, dtype: float64
Mismatch in distribution for 'AttributeLevel'.
Original distribution:
 AttributeLevel
Young      0.086332
Old        0.086332
Hoomans    0.085502
Pets       0.085502
Less       0.083888
More       0.083888
Fat        0.082395
Fit        0.082395
Male       0.081562
Female     0.081

In [35]:
import pandas as pd

# Example categorical column
column = 'ScenarioTypeStrict'

# Calculate the proportion of each category in both datasets
distribution1 = new_mme_27[column].value_counts(normalize=True).sort_index()
distribution2 = df_llm[column].value_counts(normalize=True).sort_index()

# Compare the distributions
comparison = pd.DataFrame({'Dataset1': distribution1, 'Dataset2': distribution2})
print(comparison)


                    Dataset1  Dataset2
ScenarioTypeStrict                    
Age                 0.220780  0.172664
Fitness             0.210712  0.164790
Gender              0.208586  0.163125
Social Status       0.063174  0.160640
Species             0.210384  0.171005
Utilitarian         0.086364  0.167776


In [34]:
new_mme_27.shape

(977535, 27)

In [36]:
new_mme_27.to_csv('new_mme_27.csv', index=False)

### Now doing it the other way around - the A dataset will be the new_mme_27 dataset, and the B dataset (the one to take a subset of) will be the PREP_llm_dataset.

In [ ]:
import pandas as pd
from collections import defaultdict

def create_matching_subset(dataset_a_path, dataset_b_path, chunk_size=10000):
    # Define columns to match on
    columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
               'ScenarioTypeStrict', 'Nu2mberOfCharacters', 'DiffNumberOFCharacters', 
               'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
               'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
               'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
               'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']
    
    # Load dataset A and count occurrences of each unique row configuration
    df_a = pd.read_csv(dataset_a_path, usecols=columns)
    a_counts = df_a.groupby(columns).size().to_dict()

    # Dictionary to store matched rows with the exact distribution
    matching_rows = defaultdict(list)
    final_matching_subset_b = pd.DataFrame(columns=columns)
    
    z = 0
    
    # Process B in chunks to avoid memory overload
    for chunk in pd.read_csv(dataset_b_path, chunksize=chunk_size, usecols=columns):
        z += 1

        print(f"Processing chunk...{z}")

        # Group each chunk by unique row configurations
        b_counts = chunk.groupby(columns).size().to_dict()
        
        # Loop through each unique row configuration in A
        for row_config, required_count in a_counts.items():
            if required_count <= 0:
                continue  # Skip if we've already gathered all required rows for this configuration
            
            # Check if the row configuration exists in the current chunk
            if row_config in b_counts:
                # Select matching rows from the chunk
                matching_chunk_rows = chunk[(chunk[columns] == pd.Series(row_config, index=columns)).all(axis=1)]
                
                # Select only the required number of rows
                selected_rows = matching_chunk_rows.iloc[:required_count]
                
                # Append selected rows to the final dataframe
                final_matching_subset_b = pd.concat([final_matching_subset_b, selected_rows], ignore_index=True)
                
                "print(final_matching_subset_b.shape)"
                
                # Decrease the required count
                a_counts[row_config] -= len(selected_rows)

        # Stop if we've collected enough rows to match all counts in A
        if sum(a_counts.values()) == 0:
            break
    
    return final_matching_subset_b

In [46]:
# Usage
df_a_path = 'new_mme_27.csv'
df_b_path = '../PREP_llm_dataset.csv'
new_mme_27 = create_matching_subset(df_a_path, df_b_path)

Processing chunk...1
Processing chunk...2
Processing chunk...3
Processing chunk...4
Processing chunk...5
Processing chunk...6
Processing chunk...7
Processing chunk...8
Processing chunk...9
Processing chunk...10
Processing chunk...11
Processing chunk...12
Processing chunk...13
Processing chunk...14
Processing chunk...15
Processing chunk...16
Processing chunk...17
Processing chunk...18
Processing chunk...19
Processing chunk...20
Processing chunk...21
Processing chunk...22
Processing chunk...23
Processing chunk...24
Processing chunk...25
Processing chunk...26
Processing chunk...27
Processing chunk...28
Processing chunk...29
Processing chunk...30
Processing chunk...31
Processing chunk...32
Processing chunk...33
Processing chunk...34
Processing chunk...35
Processing chunk...36
Processing chunk...37
Processing chunk...38
Processing chunk...39
Processing chunk...40
Processing chunk...41
Processing chunk...42
Processing chunk...43
Processing chunk...44
Processing chunk...45
Processing chunk...

In [48]:
new_mme_27.to_csv('distributed_llm.csv', index=False)

In [ ]:
# now distributed_llm and new_mme_27 are the exact same in terms of distribution!! :)

In [47]:
# Verify distribution
df_llm_subset = pd.read_csv(df_a_path)  # Reload A to verify distribution
for col in new_mme_27.columns:
    original_dist = df_llm_subset[col].value_counts(normalize=True)
    subset_dist = new_mme_27[col].value_counts(normalize=True)

    # Compare the distributions
    if original_dist.equals(subset_dist):
        print(f"The distribution for '{col}' matches.")
    else:
        print(f"Mismatch in distribution for '{col}'.")
        print("Original distribution:\n", original_dist)
        print("Subset distribution:\n", subset_dist)

The distribution for 'PedPed' matches.
The distribution for 'Barrier' matches.
The distribution for 'CrossingSignal' matches.
The distribution for 'AttributeLevel' matches.
The distribution for 'ScenarioTypeStrict' matches.
The distribution for 'NumberOfCharacters' matches.
The distribution for 'DiffNumberOFCharacters' matches.
The distribution for 'Man' matches.
The distribution for 'Woman' matches.
The distribution for 'Pregnant' matches.
The distribution for 'Stroller' matches.
The distribution for 'OldMan' matches.
The distribution for 'OldWoman' matches.
The distribution for 'Boy' matches.
The distribution for 'Girl' matches.
The distribution for 'Homeless' matches.
The distribution for 'LargeWoman' matches.
The distribution for 'LargeMan' matches.
The distribution for 'Criminal' matches.
The distribution for 'MaleExecutive' matches.
The distribution for 'FemaleExecutive' matches.
The distribution for 'FemaleAthlete' matches.
The distribution for 'MaleAthlete' matches.
The distrib

# ---------------------------------------------------

Test to take ALL the columns (31) ipv only the 27 :

In [50]:
df_mme_d = pd.read_csv('new_mme_27.csv')
df_llm_d = pd.read_csv('distributed_llm.csv')

In [51]:
print(df_mme_d.shape)
print(df_llm_d.shape)

(977535, 27)
(977535, 27)


In [87]:
import pandas as pd

# Example categorical column
column = 'ScenarioTypeStrict'

# Calculate the proportion of each category in both datasets
distribution1 = df_mme_d[column].value_counts(normalize=True).sort_index()
distribution2 = df_llm_d[column].value_counts(normalize=True).sort_index()

# Compare the distributions
comparison = pd.DataFrame({'mme': distribution1, 'llm': distribution2})
print(comparison)


                         mme       llm
ScenarioTypeStrict                    
Age                 0.220780  0.220780
Fitness             0.210712  0.210712
Gender              0.208586  0.208586
Social Status       0.063174  0.063174
Species             0.210384  0.210384
Utilitarian         0.086364  0.086364


In [70]:
977535*2

1955070

In [83]:
1955070/39101

50.000511495869674

In [82]:
39101/1955070
1955070-39101

1915969

In [ ]:
# checking unique values in the new_mme_27 dataset
print(df_mme_d.drop_duplicates().shape)
print(df_llm_d.drop_duplicates().shape)

# Er zijn dus 23123 unieke scenario's in de datasets

(23123, 27)
(23123, 27)


# -----------------------------------------------------------

In [93]:
import pandas as pd

def create_matching_subset_nofor(dataset_a_path, dataset_b_path, chunk_size=100_000):
    # Define columns to match on
    matching_columns = ['PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel', 
                        'ScenarioTypeStrict', 'NumberOfCharacters', 'DiffNumberOFCharacters', 
                        'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 
                        'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 
                        'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 
                        'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']
    
    # Load dataset A and count occurrences of each unique row configuration
    df_a = pd.read_csv(dataset_a_path)
    a_counts = df_a.groupby(matching_columns).size().reset_index(name="RequiredCount")
    
    # Prepare an empty DataFrame to collect the final matching subset
    final_matching_subset_b = pd.DataFrame(columns=df_a.columns)  # Use all columns from dataset A
    z = 0

    # Process dataset B in chunks
    for chunk in pd.read_csv(dataset_b_path, chunksize=chunk_size):
        z += 1
        print(f"Processing chunk...{z}")

        # Count occurrences of rows in the current chunk based on the matching columns
        b_counts = chunk.groupby(matching_columns).size().reset_index(name="AvailableCount")

        # Merge A's required counts with B's available counts (on matching columns only)
        merged_counts = pd.merge(a_counts, b_counts, on=matching_columns, how="inner")

        # Determine the number of rows to select for each configuration
        merged_counts["SelectCount"] = merged_counts[["RequiredCount", "AvailableCount"]].min(axis=1)

        # Extract matching rows from the current chunk
        for _, row in merged_counts.iterrows():
            # Create a mask for matching rows based on the matching columns
            match_condition = (chunk[matching_columns] == row[matching_columns].values).all(axis=1)
            
            # Select rows based on the mask and the required count
            selected_rows = chunk[match_condition].iloc[: int(row["SelectCount"])]
            
            # Append the matching rows (with all columns) to the final DataFrame
            final_matching_subset_b = pd.concat([final_matching_subset_b, selected_rows], ignore_index=True)
            
            # Update the remaining required counts in `a_counts`
            a_counts.loc[(a_counts[matching_columns] == row[matching_columns].values).all(axis=1), "RequiredCount"] -= row["SelectCount"]

        # Stop if all required counts are met
        if a_counts["RequiredCount"].sum() == 0:
            break

    return final_matching_subset_b


In [94]:
# Usage
df_a_path = '../PREP_llm_dataset.csv'
df_b_path = '../SharedResponses.csv'
new_mme_27 = create_matching_subset_nofor(df_a_path, df_b_path)

Processing chunk...1
Processing chunk...2
Processing chunk...3
Processing chunk...4
Processing chunk...5
Processing chunk...6
Processing chunk...7
Processing chunk...8
Processing chunk...9
Processing chunk...10
Processing chunk...11
Processing chunk...12
Processing chunk...13
Processing chunk...14
Processing chunk...15
Processing chunk...16
Processing chunk...17
Processing chunk...18


C:\Users\esmku\AppData\Local\Temp\ipykernel_19920\2220529611.py:21: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(dataset_b_path, chunksize=chunk_size):


Processing chunk...19
Processing chunk...20
Processing chunk...21
Processing chunk...22
Processing chunk...23
Processing chunk...24
Processing chunk...25
Processing chunk...26
Processing chunk...27
Processing chunk...28
Processing chunk...29


KeyboardInterrupt: 